Loading Bert and Applying to dataset


The BERT pre-trained model we will use will be CASED to preserve casing as that likely contains important information for our application.

<br>
1/14/21
Initially, the plan will be to go with Bert Uncased Large model. We will use the Whole word masking version released in 2019 initially. 

References: https://github.com/chambliss/Multilingual_NER/blob/master/notebooks/exploratory_nbs/exploring_multilingual_BERT.ipynb

https://gab41.lab41.org/how-to-fine-tune-bert-for-named-entity-recognition-2257b5e5ce7e

https://towardsdatascience.com/unsupervised-ner-using-bert-2d7af5f90b8a

Dependenccies:
Will be using huggingface's pretrained models here https://github.com/huggingface/transformers

https://www.tensorflow.org/tutorials/text/classify_text_with_bert

https://stackoverflow.com/questions/58636587/how-to-use-bert-for-long-text-classification

In [2]:
import sqlite3
# Connect to db
def connect_to_db():
    database = "../db/sentences.db"
    conn = create_connection(database)
    return conn
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Exception as e:
        print(e)
 
    return None

# Get sentences
def get_sentences_from_db(conn):
    sentences = ''
    cur = conn.cursor()
    cur.execute("select processed_sentence, haspolys from sentences") 
    rows = cur.fetchall()
    return rows
    

First, create a connection to the database and retrieve the sentences and whether they contain a polymer

In [3]:
conn = connect_to_db()
sentences = get_sentences_from_db(conn)

Create a dataframe with 2 columns, the sentence as the truth.

In [4]:
import pandas as pd
df = pd.DataFrame(columns=["sentence","contains_polymer"])

df["sentence"] = [pair[0] for pair in sentences]
df["contains_polymer"] = [pair[1] for pair in sentences]

Set up BERT

In [71]:
!pip install bert-for-tf2
!pip install sentencepiece

In [1]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
import pandas as pd
import numpy as np

In [11]:
y = df.sentence

In [5]:
# CREATING the bert tokenizer
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [8]:
def tokenize_sentences(sentences):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences))

In [32]:
tokenized_sentences = [tokenize_sentences(sentence) for sentence in df["sentence"]]


In [54]:
sentences_with_len = [[sentence, y[i], len(sentence)]
                 for i, sentence in enumerate(tokenized_sentences)]

sentences_with_len = [x for x in sentences_with_len if x[2] != 0]
sentences_with_len.sort(key=lambda x: x[2])
sorted_sentences_labels = [(sentence_lab[0], sentence_lab[1]) for sentence_lab in sentences_with_len]


In [47]:
tokenized_sentences[:3]

[[156,
  26426,
  5864,
  22258,
  8690,
  4434,
  12120,
  3101,
  17268,
  17129,
  1183,
  113,
  181,
  118,
  2495,
  5822,
  3269,
  114,
  851,
  2101,
  23415,
  113,
  3084,
  18873,
  7582,
  176,
  1193,
  2528,
  1233,
  114,
  12120,
  27467,
  3291,
  23043,
  26911,
  1197,
  2352],
 [26446, 10922, 117, 12068, 7014, 117, 16749, 23618, 15463, 115],
 [1426,
  7443,
  8891,
  17129,
  26911,
  1197,
  185,
  7889,
  5053,
  1665,
  10847,
  117,
  11497,
  17143,
  1179,
  2024,
  6058,
  10847,
  117,
  1922,
  2127,
  4052,
  117,
  11497,
  17143,
  1179,
  21251,
  20581,
  117,
  153,
  119,
  155,
  119,
  1975]]

In [55]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_sentences_labels, output_types=(tf.int32, tf.int32))

In [56]:
processed_dataset

<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>

In [57]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [58]:
import math 

TOTAL_BATCHES = math.ceil(len(sorted_sentences_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [59]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [60]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [61]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [62]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [63]:
text_model.fit(train_data, epochs=NB_EPOCHS)


Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ResourceExhaustedError: OOM when allocating tensor with shape[28996,200] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:Mul]

In [22]:
train_data

<SkipDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [44]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased')
encoded_input = tokenizer(sentences[0])
print(encoded_input)

{'input_ids': [101, 156, 26426, 5864, 22258, 8690, 4434, 12120, 3101, 17268, 17129, 1183, 113, 181, 118, 2495, 5822, 3269, 114, 851, 2101, 23415, 113, 3084, 18873, 7582, 176, 1193, 2528, 1233, 114, 12120, 27467, 3291, 23043, 26911, 1197, 2352, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Sentence training classification.

The following model will be explored for whole sentence classification
https://huggingface.co/distilbert-base-uncased

In [31]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
unmasker(sentences)

Token indices sequence length is longer than the specified maximum sequence length for this model (984 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:73] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1154567725056 bytes. Buy new RAM!